# Multi-GPU K-Means with Dask

In this notebook you will use GPU-accelerated K-means to identify population clusters in a multi-node, multi-GPU scalable way with Dask.

## Objectives

By the time you complete this notebook you will be able to:

- Use distributed, GPU-accelerated K-means with Dask

## Imports

First we import the needed modules to create a Dask cuDF cluster.

In [1]:
import subprocess
import logging

from dask.distributed import Client, wait, progress
from dask_cuda import LocalCUDACluster

After that, we create the cluster.

In [2]:
cmd = "hostname --all-ip-addresses"
process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
output, error = process.communicate()
IPADDR = str(output.decode()).split()[0]

cluster = LocalCUDACluster(ip=IPADDR, silence_logs=logging.ERROR)
client = Client(cluster)

distributed.preloading - INFO - Import preload module: dask_cuda.initialize
distributed.preloading - INFO - Import preload module: dask_cuda.initialize
distributed.preloading - INFO - Import preload module: dask_cuda.initialize
distributed.preloading - INFO - Import preload module: dask_cuda.initialize


Finally, as we did before, we import CUDA context creators after setting up the cluster so they don't lock to a single device.

In [3]:
import cudf
import dask_cudf

import cuml
from cuml.dask.cluster import KMeans

## Load and Persist Data

We will begin by loading the data, The data set has the two grid coordinate columns, `easting` and `northing`, derived from the main population data set we have prepared.

In [4]:
ddf = dask_cudf.read_csv('./data/pop5x_2-07.csv', dtype=['float32', 'float32'])

Training the K-means model is very similar to both the scikit-learn version and the cuML single-GPU version--by setting up the client and importing from the `cuml.dask.cluster` module, the algorithm will automatically use the local Dask cluster we have set up.

Note that calling `.fit` triggers Dask computation.

In [5]:
%%time
dkm = KMeans(n_clusters=20)
dkm.fit(ddf)

CPU times: user 4.91 s, sys: 6.16 s, total: 11.1 s
Wall time: 4min 14s


Once we have the fit model, we extract the cluster centers and rename the columns from their generic '0' and '1' to reflect the data on which they were trained.

In [7]:
cluster_centers = dkm.cluster_centers_
cluster_centers.columns = ddf.columns
cluster_centers

,northing,easting
0,-1.229991e+06,558353.81250
1,2.655240e+06,468942.65625
2,6.369926e+05,539382.00000
3,-5.343192e+06,646409.93750
4,3.486508e+06,422979.50000
5,-3.034439e+06,608732.37500
6,-1.570282e+05,561244.37500
7,-9.711531e+05,559422.93750
8,1.601888e+06,509016.21875
9,-1.691054e+06,560515.00000


## Exercise: Count Members of the Southernmost Cluster

Using the `cluster_centers`, identify which cluster is the southernmost (has the lowest `northing` value) with the `nsmallest` method, then use `dkm.predict` to get labels for the data, and finally filter the labels to determine how many individuals the model estimated were in that cluster.

In [21]:
south = cluster_centers.nsmallest(1, 'northing').index[0]
print(south)
lables = dkm.predict(ddf)
lables[lables==south].compute().shape[0]

3


20512372

In [13]:
# %load solutions/southernmost_cluster
south_idx = cluster_centers.nsmallest(1, 'northing').index[0]
labels_predicted = dkm.predict(ddf)
labels_predicted[labels_predicted==south_idx].compute().shape[0]


<br>
<div align="center"><h2>Please Restart the Kernel</h2></div>

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Next

In the next notebook, you will calculate infection risk again, this time using the powerful XGBoost algorithm.